<a href="https://colab.research.google.com/github/amirhoseinaghaei/Research_Simulation/blob/main/Research_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
# import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque
import torch.optim as optim
# from nn import LSTMLayer
from torch.nn import MSELoss as MSELoss

In [2]:
class State(object):
    def __init__(self , Name, Au , Ch, BT, Ra, U):
        self.Name = Name
        self.Au = Au
        self.Ch = Ch
        self.BT = BT
        self.Ra = Ra
        self.U = U


In [3]:
class DefiningWirelessChannels(object):
    def __init__(self, NUM_Ch , Rate_List):
        self.NUM_Ch = NUM_Ch
        self.Rate_List = Rate_List
        self.Rate_Dict = {}
    def Create_RateDict(self):
        for i in range(self.NUM_Ch):
            self.Rate_Dict[f"Ch{i+1}"] = self.Rate_List[i]
            

In [4]:

import copy
import random
import numpy as np
import torch

rnd_seed = 1
torch.manual_seed(rnd_seed)
np.random.seed(rnd_seed)

lb_size = 2048
n_lstm = 16
max_time = 50
policy_lr = 0.1
lstm_lr = 1e-2
l2_regularization = 1e-6
avg_window = 750

class Environment(object):
    
    def __init__(self, B_max, C_max , WirelessChannelClass = DefiningWirelessChannels(2,[5,20]), K = 3 , W = 30 , Fc = 5):
        
        self.inital_state = None
        self.state = None
        self.updated = True
        self.update = None
        self.deadline = 25
        self.time = None
        self.Wait = None
        self.Fc = Fc
        self.K = K
        self.W = W
        self.B_max = B_max
        self.C_max = C_max
        self.WirelessChannelClass = WirelessChannelClass
        self.WirelessChannelClass.Create_RateDict()
        self.ConfigureParameters()
    
    def ConfigureParameters(self):
        self.BT_max = self.C_max + self.B_max
        self.Au_min = (self.B_max/max(self.WirelessChannelClass.Rate_List)) + (self.C_max/self.Fc)
        self.Au_max = self.W + self.W*self.K + (self.B_max/min(self.WirelessChannelClass.Rate_List)) + (self.C_max/self.Fc)
        self.ADT_max = (self.B_max/min(self.WirelessChannelClass.Rate_List)) + (self.C_max/self.Fc)

    def CreateStates(self):
        D = self.ADT_max - self.Au_min
        Deadline = 25
        self.StateList = []
        for i in range(int(self.Au_min),int(self.Au_max)+1):
            if self.Au_min <= i <= self.W:
                BT = 0
                if self.Au_min <= i <= self.ADT_max + self.Au_min -1:
                    U_max = i - self.Au_min
                self.StateList.append(State(f"({i}, Ch1, {BT}, {1}, {0})", Au= i, Ch ="Ch1" , BT = BT, Ra = 1, U = 0))
                self.StateList.append(State(f"({i}, Ch2, {BT}, {1}, {0})", Au= i, Ch ="Ch2" , BT = BT, Ra = 1, U = 0))
                for j in range(0,int(U_max)+1):
                    self.StateList.append(State(f"({i}, Ch1, {BT}, {0}, {j})", Au= i, Ch ="Ch1" , BT = BT, Ra = 0, U = j))
                    self.StateList.append(State(f"({i}, Ch2, {BT}, {0}, {j})", Au= i, Ch ="Ch2" , BT = BT, Ra = 0, U = j))
            if i > self.W and 0 < i%self.W <= self.Au_min:
                first = self.BT_max - (i%self.W)*max(self.WirelessChannelClass.Rate_List)
                
                if first < self.C_max : 
                   
                    BT = []
                    Len = i%self.W- self.B_max/max(self.WirelessChannelClass.Rate_List)
                    Len = int(Len)
                    for j in range(Len+1):
                        BT.append(self.C_max-(j)*self.Fc)
                    BT.sort()
                   
                else:
                    BT = [first]
                last = self.BT_max - (i%self.W)*min(self.WirelessChannelClass.Rate_List)
                crowler = first
                while crowler != last:
                    BT.append(int(crowler + D)) if crowler + D > 25 else None
                    crowler = crowler + D
                if self.W <= i < Deadline + self.Au_min :
                    U_max = i - self.Au_min
                    for bt in BT:
                        self.StateList.append(State(f"({i}, Ch1, {bt}, {1}, {0})", Au= i, Ch ="Ch1" , BT = bt, Ra = 1, U = 0))
                        self.StateList.append(State(f"({i}, Ch2, {bt}, {1}, {0})", Au= i, Ch ="Ch2" , BT = bt, Ra = 1, U = 0))
                        for j in range(0,int(U_max)+1):
                            self.StateList.append(State(f"({i}, Ch1, {bt}, {0}, {j})", Au= i, Ch ="Ch1" , BT = bt, Ra = 0, U = j))
                            self.StateList.append(State(f"({i}, Ch2, {bt}, {0}, {j})", Au= i, Ch ="Ch2" , BT = bt, Ra = 0, U = j))
                else: 
                    U_max = 25 
                    for bt in BT:
                        self.StateList.append(State(f"({i}, Ch1, {bt}, {1}, {0})", Au= i, Ch ="Ch1" , BT = bt, Ra = 1, U = 0))
                        self.StateList.append(State(f"({i}, Ch2, {bt}, {1}, {0})", Au= i, Ch ="Ch2" , BT = bt, Ra = 1, U = 0))
                        for j in range(0,int(U_max)+1):
                            self.StateList.append(State(f"({i}, Ch1, {bt}, {0}, {j})", Au= i, Ch ="Ch1" , BT = bt, Ra = 0, U = j))
                            self.StateList.append(State(f"({i}, Ch2, {bt}, {0}, {j})", Au= i, Ch ="Ch2" , BT = bt, Ra = 0, U = j))
            if i> self.C_max and i%self.W > self.Au_min and i%self.W < self.B_max/min(self.WirelessChannelClass.Rate_List):
                BT = [0,5,10,15,20,25]
                first = self.BT_max - (i%self.W)*max(self.WirelessChannelClass.Rate_List)
                if first <= 25 : 
                    BT = [0,5,10,15,20,25]
                else:
                    BT = [first]
                last = self.BT_max - (i%self.W)*min(self.WirelessChannelClass.Rate_List)
                crowler = first
                while crowler != last:
                    BT.append(int(crowler + D)) if crowler + D > 25 else None
                    crowler = crowler + D           
                U_max = 25 
                for bt in BT:
                    self.StateList.append(State(f"({i}, Ch1, {bt}, {1}, {0})", Au= i, Ch ="Ch1" , BT = bt, Ra = 1, U = 0))
                    self.StateList.append(State(f"({i}, Ch2, {bt}, {1}, {0})", Au= i, Ch ="Ch2" , BT = bt, Ra = 1, U = 0))
                    for j in range(0,int(U_max)+1):
                        self.StateList.append(State(f"({i}, Ch1, {bt}, {0}, {j})", Au= i, Ch ="Ch1" , BT = bt, Ra = 0, U = j))
                        self.StateList.append(State(f"({i}, Ch2, {bt}, {0}, {j})", Au= i, Ch ="Ch2" , BT = bt, Ra = 0, U = j))
            if i> self.C_max and i%self.W > self.Au_min and i%self.W >= self.B_max/min(self.WirelessChannelClass.Rate_List):
                BT = [0]
                crowler = 0
                if int(self.C_max - (i%self.W - ((self.B_max)/min(self.WirelessChannelClass.Rate_List)))*self.Fc) > 0:
                    while crowler != int(self.C_max - (i%self.W - ((self.B_max)/min(self.WirelessChannelClass.Rate_List)))*self.Fc):
                        BT.append(crowler + self.Fc)  
                        crowler = crowler + self.Fc             
                U_max = 25 
                for bt in BT:
                    self.StateList.append(State(f"({i}, Ch1, {bt}, {1}, {0})", Au= i, Ch ="Ch1" , BT = bt, Ra = 1, U = 0))
                    self.StateList.append(State(f"({i}, Ch2, {bt}, {1}, {0})", Au= i, Ch ="Ch2" , BT = bt, Ra = 1, U = 0))
                    for j in range(0,int(U_max)+1):
                        self.StateList.append(State(f"({i}, Ch1, {bt}, {0}, {j})", Au= i, Ch ="Ch1" , BT = bt, Ra = 0, U = j))
                        self.StateList.append(State(f"({i}, Ch2, {bt}, {0}, {j})", Au= i, Ch ="Ch2" , BT = bt, Ra = 0, U = j))

        self.initial_State  = []
        for i in self.StateList:
            if i.Ra == 1:
                self.initial_State.append(i)   

    def reset_state(self):
        self.state = random.choice(self.initial_State)
        self.inital_state = copy.deepcopy(self.state)
        return np.array([self.state.Au, self.state.Ch, self.state.BT, self.state.Ra, self.state.U])

    def reset_paramter(self):
        self.Wait = 0
        self.time = 0
        self.updated = False
    def remained_BT_modification(self):
        if self.state.Au % self.W < self.W-1 and self.state.BT == 0:
            self.state.BT = 0
        elif self.state.Au % self.W == self.W-1:
            self.state.BT = self.BT_max
        else:
            if self.C_max < self.state.BT <= self.BT_max:
                self.state.BT -= self.WirelessChannelClass.Rate_Dict[self.state.Ch]
            else:
                self.state.BT -= self.Fc
    def wireless_channel_modification(self):
        random_generated = random.uniform(0, 1)
        if random_generated < 0.5:
            self.state.Ch = "Ch1"
        else:
            self.state.Ch = "Ch2"
    def request_pending_time_modification(self,action):
        if action == 1:
            self.state.U = 0
        elif action == 0 and self.Sendback:
            self.state.U = 0
        else: 
            self.state.U += 1

    def AoI_modification(self, action):
        if self.Wait == True:  
            if self.update == False:
                self.state.Au += 1
            elif self.update and action == 1:
                self.state.Au += 1
            elif self.update and action == 0:
                self.state.Au = self.state.Au % self.W + 1 
        else:
            self.updated = True
            if self.inital_state.BT == 0:
                self.state.Au = (self.state.Au%self.W) + 1
            else:
                self.state.Au = self.W + (self.state.Au%self.W) + 1
            self.Wait = True
    def state_transition(self, action):
            self.remained_BT_modification()
            self.wireless_channel_modification()
            self.request_pending_time_modification(action)
            self.state.Ra = 0
            self.AoI_modification(action)
            self.state.Name = f"({self.state.Au}, {self.state.Ch}, {self.state.BT}, {self.state.Ra}, {self.state.U})" 

    def step(self, action):
        reward = 0
        if self.time == 0:
            self.Wait = action == 0
            self.Sendback = action == 1
        self.time += 1
        done = self.time == self.deadline
        self.update = self.state.BT == self.Fc
        if self.update and self.updated == False and self.Wait:
            print("updated")
            self.updated = True
            if self.inital_state.Au >= self.W and self.inital_state.BT != 0:
                    reward += self.W + (self.inital_state.Au)%(self.W)
            else:
                reward += (self.inital_state.Au)%(self.W)
            reward -= (self.time + (self.state.Au)%(self.W) + 1)
        
        if done and self.Wait and self.updated == False:
            reward += (self.inital_state.Au)%(self.W)
            reward -= (self.time + ((self.state.Au + 1)%self.W + self.W))
        self.state_transition(action)
        
        return np.array([self.state.Au, self.state.Ch, self.state.BT, 0, self.state.U ]) , reward , done


In [41]:
from torch.nn.modules.rnn import LSTM
def to_one_hot(y, n_dims=None):
    """ Take integer y (tensor or variable) with n dims and convert it to 1-hot representation with n+1 dims. """
    y_tensor = y.data if isinstance(y, Variable) else y
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    n_dims = n_dims if n_dims is not None else int(torch.max(y_tensor)) + 1
    y_one_hot = torch.zeros(y_tensor.size()[0], n_dims).scatter_(1, y_tensor, 1)
    y_one_hot = y_one_hot.view(*y.shape, -1)
    return Variable(y_one_hot) if isinstance(y, Variable) else y_one_hot

class RRLSTM(nn.Module):
    def __init__(self, state_input_size, n_actions, buffer, n_units, lstm_lr, l2_regularization,
                 return_scaling, lstm_batch_size=128, continuous_pred_factor=0.5):
        super(RRLSTM, self).__init__()
        self.buffer = buffer
        self.return_scaling = return_scaling
        self.lstm_batch_size = lstm_batch_size
        self.continuous_pred_factor = continuous_pred_factor
        self.n_actions = n_actions    
        # Forget gate and output gate are deactivated as used in the Atari games, see Appendix S4.2.1
        
        self.lstm = LSTM(state_input_size + n_actions, n_units)
        self.linear = nn.Linear(n_units, 1)
        self.optimizer = optim.Adam(self.parameters(), lr=lstm_lr, weight_decay=l2_regularization)
        self.lstm_updates = 0
    def forward(self, input):
        print(input)
        states, actions = input
        # # Prepare input features
        # repaired = states[:, :, 0:1]
        # # print(f"input: {input}")
        # # print(f"repaired: {repaired}")
        # transport_cond = states[:, :, 1:3]
        # # print(f"transport_cond: {transport_cond}")
        # brands = to_one_hot(states[:, :, 3], 4)
        # # print(f"brands: {brands}")
        # time = states[:, :, 4:] / states.shape[1]
        # # print(f"brands: {states[:, :, 4:]}")
        # states = torch.cat([repaired, transport_cond, brands, time], 2)
        actions = to_one_hot(actions, self.n_actions)
        actions = torch.cat((actions, torch.zeros((actions.shape[0], 1, self.n_actions))), 1)
        # print(f"Hi{actions}")
        # print(f"Hi{states}")
        print(f"First{states.size()}")
        print(f"Second{actions.size()}")
        input = torch.cat((states, actions), 2)
        # Run the lstm
        lstm_out = self.lstm.forward(input, return_all_seq_pos=True)
        return self.linear(lstm_out[0])
    def redistribute_reward(self, states, actions):
        # Prepare LSTM inputs
        states_var = Variable(torch.FloatTensor(states)).detach()
        delta_states = torch.cat([states_var[:, 0:1, :], states_var[:, 1:, :] - states_var[:, :-1, :]], dim=1)
        actions_var = Variable(torch.FloatTensor(actions)).detach()
        # Calculate LSTM predictions
        lstm_out = self.forward([delta_states, actions_var])
        pred_g0 = torch.cat([torch.zeros_like(lstm_out[:, 0:1, :]), lstm_out], dim=1)[:, :-1, :]
        # Difference of predictions of two consecutive timesteps.
        redistributed_reward = pred_g0[:, 1:, 0] - pred_g0[:, :-1, 0]
        new_reward = redistributed_reward * self.return_scaling
        return new_reward

    # Trains the LSTM until -on average- the main loss is below 0.25.
    def train(self, episode):

        i = 0
        loss_average = 0.3
        mse_loss = MSELoss(reduction="none")
        while loss_average > 0.15:
            i += 1
            self.lstm_updates += 1
            self.optimizer.zero_grad()

            # Get samples from the lesson buffer and prepare them.
            states, actions, rewards, lenght = self.buffer.sample(self.lstm_batch_size)
            lenght = lenght[:, 0]
            states_var = Variable(torch.FloatTensor(states)).detach()
            actions_var = Variable(torch.FloatTensor(actions)).detach()
            rewards_var = Variable(torch.FloatTensor(rewards)).detach()

            # Scale the returns as they might have high / low values.
            returns = torch.sum(rewards_var, 1, keepdim=True) / self.return_scaling

            # Calculate differences of states
            delta_states = torch.cat([states_var[:, 0:1, :], states_var[:, 1:, :] - states_var[:, :-1, :]], dim=1)
            print(f"HIIIII{delta_states}")
            # Run the LSTM
            lstm_out = self.forward([delta_states, actions_var])
            predicted_G0 = lstm_out.squeeze()

            # Loss calculations
            all_timestep_loss = mse_loss(predicted_G0, returns.repeat(1, predicted_G0.size(1)))

            # Loss at any position in the sequence
            aux_loss = self.continuous_pred_factor * all_timestep_loss.mean()

            # LSTM is mainly trained on getting the final prediction of g0 right.
            main_loss = all_timestep_loss[range(self.lstm_batch_size), lenght[:] - 1].mean()

            # LSTM update and loss tracking
            lstm_loss = main_loss + aux_loss
            lstm_loss.backward()
            loss_np = lstm_loss.data.numpy()
            main_loss_np = main_loss.data.numpy()
            loss_average -= 0.01 * (loss_average - main_loss_np)
            if main_loss_np > loss_average * 2:
                loss_average = loss_np
            self.optimizer.step()

In [5]:
class LessonBuffer:
    def __init__(self, size, deadline, state_variables):
        self.size = size 
        self.states_buffer = np.empty(shape=(size, deadline + 1, state_variables))
        self.actions_buffer = np.empty(shape=(size, deadline))
        self.rewards_buffer = np.empty(shape=(size, deadline))
        self.lens_buffer = np.empty(shape=(size, 1), dtype=np.int32)
        self.next_spot_to_add = 0
        self.buffer_is_full = False
        self.samples_since_last_training = 0       
    def different_returns_encountered(self):
        if self.buffer_is_full:
            # print(np.unique(self.rewards_buffer[..., -1]).shape[0] > 1)
            return np.unique(self.rewards_buffer[..., -1]).shape[0] > 1
        else:
            return np.unique(self.rewards_buffer[:self.next_spot_to_add, -1]).shape[0] > 1
    def full_enough(self):
        return self.buffer_is_full or self.next_spot_to_add > 256  
    def add(self,states, actions, rewards):
        traj_length = states.shape[0]
        print(traj_length)
        next_ind = self.next_spot_to_add
        self.next_spot_to_add = self.next_spot_to_add + 1
        if self.next_spot_to_add >= self.size:
            self.buffer_is_full = True
        self.next_spot_to_add = self.next_spot_to_add % self.size
        # print(states)
        # print(states.squeeze())
        self.states_buffer[next_ind, :traj_length] = states.squeeze()
        self.states_buffer[next_ind, traj_length:] = 0
        self.actions_buffer[next_ind, :traj_length - 1] = actions
        self.actions_buffer[next_ind, traj_length:] = 0
        self.rewards_buffer[next_ind, :traj_length - 1] = rewards
        self.rewards_buffer[next_ind, traj_length:] = 0
        # self.lens_buffer[next_ind] = traj_length
    def sample(self, batch_size):
        self.samples_since_last_training = 0
        if self.buffer_is_full: 
            indices = np.random.randint(0, self.size, batch_size)
        else: 
            indices = np.random.randint(0, self.next_spot_to_add, batch_size)
        return (self.states_buffer[indices, :, :], self.actions_buffer[indices, :],
                self.rewards_buffer[indices, :], self.lens_buffer[indices, :])


In [ ]:
Model = Environment(100,25)
Model.CreateStates()
Lesson_buffer = LessonBuffer(1000, 25, 5)
episode = 0

class Actor():
    def __init__(self, environment, lr):
        self.environment = environment
        self.lr = lr 
        self.state = None
    def act(self):
        if self.environment.state.Ra == 0 and self.environment.state.U == 0:
            action = 0
        elif self.environment.state.Ra == 1 and self.environment.state.U == 0:
            action = 1 if  random.uniform(0, 1) < 0.5 else 0
        elif self.environment.state.Ra == 0 and self.environment.state.U == 24:
            action = 1
        elif self.environment.state.U > 0:
            action = 0

        self.state , reward , done = self.environment.step(action)
        return self.state, action, reward, done
actor = Actor(Model, 0.1)
# actor.act()
rudder_lstm = RRLSTM(state_input_size=5, n_actions= 2, buffer=Lesson_buffer, n_units=n_lstm,
                       lstm_lr=lstm_lr, l2_regularization=l2_regularization, return_scaling=10,
                       lstm_batch_size=8, continuous_pred_factor=0.5)

for i in range(1000):
    episode += 1
    actor.environment.reset_paramter()
    state = actor.environment.reset_state()
    rewards = []
    states = [state]
    actions = []
    done = False
    while not done:
        state, action, reward, done = actor.act()
        actions.append(action)
        states.append(state)
        rewards.append(reward)
        if done: 
            for i in states: 
                if i[1] == "Ch1":
                    i[1] = 1
                else:
                    i[1] = 0
            states = np.stack(states)
            rewards = np.array(rewards)
            actions = np.array(actions)
            Lesson_buffer.add(states = states, actions = actions, rewards = rewards)
            # print(actions)
            # print(states)
            # print(rewards)
            if  Lesson_buffer.full_enough():
              # print("True")
            # If RUDDER is run, the LSTM is trained after each episode until its loss is below a threshold.
            # Samples will be drawn from the lessons buffer.
              if episode % 25 == 0:
                print("READY FOR TRAIN")
                rudder_lstm.train(episode=episode)